# Yelp API - Lab


## Part 1 - Set up

In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
import sqlalchemy
import config
config.database = 'yelp'

In [39]:
sql_alchemy_connection = f'''mysql+mysqlconnector://{config.user}:{config.password}@{config.host}:{config.port}/{config.database}'''

db = sqlalchemy.create_engine(sql_alchemy_connection, echo = False, connect_args = {'ssl_disabled' : True})

### Define a new schema, yelp

Only run this once (aka never)

In [37]:
# create_schema = '''CREATE SCHEMA yelp;'''

## Part 2 -  Get Business Data

Safe to run multiple times.
Change reset to True to drop the table and re-pull from the API


In [30]:
business_table = '''
    CREATE TABLE IF NOT EXISTS yelp.business_alchemy (
          table_id            int
                              NOT NULL
                              UNIQUE
                              AUTO_INCREMENT
                              PRIMARY KEY                  ,

          yelp_id             varchar(200)                 ,
          alias               varchar(200)                 ,
          name                varchar(200)                 ,
          review_count        int                          ,
          rating              int                          ,
          location_address1   varchar(200)                 ,
          location_address2   varchar(200)                 ,
          location_address3   varchar(200)                 ,
          location_city       varchar(200)                 ,
          location_zip_code   varchar( 10)                 ,
          location_country    varchar(200)                 ,
          location_state      varchar(200)                 ,
          price               float
    )  ENGINE=INNODB;
'''

In [40]:
# with db.connect() as conn:
#     conn.execute('''DROP TABLE IF EXISTS yelp.business_alchemy''')
#     conn.execute(business_table)

In [41]:
# URL Parameter Builder
def package_business_request(term = 'mediterranean', 
                             location = 'chelsea, NY',
                             limit = 50,
                             offset = 0):
    return {'term'     : term,
            'location' : location,
            'limit'    : limit,
            'offset'   : offset}
business_search_params_builder = lambda offset : package_business_request(offset = offset)

# Yelp API Endpoint for Business Search
businesses_url = 'https://api.yelp.com/v3/businesses/search'

In [43]:
import utilities

In [46]:
response = utilities.yelp_call(businesses_url, business_search_params_builder(0), config.api_key)

In [47]:
df = utilities.parse_business_response(response)

In [52]:
df.to_sql('business_alchemy', engine, if_exists = 'append', index = False)

In [53]:
df.to_sql('business_alchemy_new', engine, if_exists = 'append', index = False)